In [38]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib
import random
from datetime import datetime
from geopy.geocoders import Nominatim


In [39]:
# %pip install faker
# %pip install pandas
# %pip install numpy
# %pip install openpyxl
# %pip install --upgrade nbformat
# %pip install apache-airflow
# %pip install geopy
# %pip install papermill


In [40]:
# def get_location_details(place_name):
#     geolocator = Nominatim(user_agent="geoapiExercises")
#     location = geolocator.geocode(place_name)
#     if location:
#         return location.latitude, location.longitude, location.raw
#     else:
#         return None, None, None

# # Exemplo de uso
# place_name = "Bonsucesso"
# lat, lon, details = get_location_details(place_name)

# if lat and lon:
#     print(f"Latitude e Longitude de {place_name}: {lat}, {lon}")
#     print(f"Detalhes: {details}")
#     state = details.get('address', {}).get('state', 'Estado não encontrado')
#     country = details.get('address', {}).get('country', 'País não encontrado')
#     print(f"Estado: {state}")
#     print(f"País: {country}")
# else:
#     print(f"Não foi possível encontrar a localização de {place_name}.")


In [41]:
# geolocator = Nominatim(user_agent="geoapiExercises")
# location = geolocator.geocode('Bonsucesso')
# location.address


In [42]:
# Dicionário para armazenar os DREs gerados por ano e semestre
dre_registro = {}

def gerar_dre(periodo_ingresso_ufrj):
    periodo, semestre = periodo_ingresso_ufrj.split('/')
    periodo = periodo[-2:]
    semestre = str(int(semestre)-1)
    
    chave = f'{periodo}{semestre}'
    if chave not in dre_registro:
        dre_registro[chave] = set()
    
    while True:
        ultimos_digitos = f'{random.randint(0, 99999):05d}'
        dre = f'1{periodo}{semestre}{ultimos_digitos}'
        if ultimos_digitos not in dre_registro[chave]:
            dre_registro[chave].add(ultimos_digitos)
            return dre

In [43]:
def calcular_idade_ingresso_ufrj(row):
    """
    Calcula a idade atual com base na data de nascimento fornecida.
    
    Args:
    data_nascimento (str): Data de nascimento no formato 'dd/mm/yyyy'.
    
    Returns:
    int: Idade atual em anos, ou None se a data estiver em um formato inválido.
    """
    data_nascimento = row['dataNascimento']
    periodo_ingresso = row['periodoIngressoCursoAtual']
    try:
        # Converter a string da data de nascimento em um objeto datetime
        ano_nascimento = datetime.strptime(data_nascimento, '%d/%m/%Y').year

        # Obter a data atual
        ano_ingresso = int(periodo_ingresso.split('/')[0])

        # Calcular a idade
        idade = ano_ingresso - ano_nascimento
        return idade
    except ValueError:
        # Retornar None se a data de nascimento estiver em um formato inválido
        return None

# Processamento


Algumas colunas não trazem ganho de informação, como por exemplo reprovações já estão na coluna disciplinas cursadas, e periodo com cr menor que 3, pode ser analisar na coluna CR por periodo


In [44]:
# Lê o arquivo Excel
df = pd.read_parquet("../../database/bronze/alunos_ufrj.parquet")

In [30]:
# Inicializa o Faker para gerar nomes em português do Brasil
faker = Faker("pt_BR")

# Gera uma lista de nomes completos para cada linha do DataFrame
df["nomeCompleto"] = [faker.name() for _ in range(df.shape[0])]

df["matriculaDre"] = df['periodoIngressoUFRJ'].apply(gerar_dre)

In [31]:
cod_cursos_computacao = [3101020000, 3101070000, 3109000100]
df = df[df["codCursoAtual"].isin(cod_cursos_computacao)]

In [32]:
mapeamento_genero = {
    'M': 'Masculino',
    'F': 'Feminino'
}
df['sexo'] = df['sexo'].map(mapeamento_genero)

In [33]:
df['idade'] = df.apply(calcular_idade_ingresso_ufrj, axis=1)

In [36]:
df.to_parquet('../../database/silver/alunos_ufrj.parquet')

### Disciplinas Cursadas


Nessa seção vamos separar a lista de disciplinas em varias colunas, uma para cada disciplina


In [ ]:
def processar_disciplina(disciplina):
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)

In [ ]:
# Converte a coluna 'disciplinasCursadas' para string
df["disciplinasCursadas"] = df["disciplinasCursadas"].astype(str)

# Processa a coluna 'disciplinasCursadas' para separar em periodo, disciplina, grau, stituação
df["disciplinasCursadas"] = df["disciplinasCursadas"].apply(lambda row: [processar_disciplina(item) for item in row.split("\n")])

# Explode a coluna 'disciplinasCursadas' para dividir as listas em linhas separadas
df_exploded = df.explode("disciplinasCursadas")

Agora vamos pegar as informações de cada desciplina e desmembrar em novas colunas


In [ ]:
def process_disciplinas(x):
    if x[0] == 'nan':
        # return (np.nan, np.nan, np.nan, np.nan, np.nan)
        return np.nan
    try:
        return (
            x[0].replace(" ", ""),
            x[1].split()[0].replace(" ", ""),
            " ".join(x[1].split()[1:]),
            x[2].replace(" ", ""),
            x[3].replace(" ", ""),
        )
    except IndexError:
        print("Erro de índice para:", x)
        return None

In [ ]:
df_exploded["disciplinasCursadas"] = df_exploded["disciplinasCursadas"].apply(process_disciplinas)

In [ ]:
df_disciplina = pd.DataFrame(
    df_exploded["disciplinasCursadas"].tolist(),
    index=df_exploded.index,
    columns=(
        "periodo",
        "cod_disciplina",
        "disciplina",
        "grau_disciplina",
        "situacao_disciplina",
    ),
)
df_disciplina[["matriculaDRE"]] = df_exploded[["matriculaDRE"]]
df_disciplina.dropna(inplace=True)

## Meger Colunas Tratadas


In [ ]:
merged_df = df_craPeriodo.merge(df_crPeriodo, how='outer')
merged_df = merged_df.merge(df_disciplina, how='outer')

In [ ]:
df.drop(['periodo_temp'], inplace=True, axis=1)
df_completo = pd.merge(merged_df, df, on=["matriculaDRE"], how="left")

In [ ]:
df_completo[["ano", "semestre"]] = df_completo["periodo"].str.split("/", expand=True)

In [ ]:
df_completo.drop(
    ["disciplinasCursadas", "crPorPeriodo", "craPorPeriodo"], axis=1, inplace=True
)

### Padronizando Nomeclatura das Colunas


In [ ]:
# colunas = {
#     "cod_disciplina": "cod_disciplina",
#     "disciplina": "nome_disciplina",
#     "matriculaDRE": "matricula_dre",
#     "dataNascimento": "data_nascimento",
#     "periodoIngressoUFRJ": "periodo_ingresso_ufrj",
#     "cursoIngressoUFRJ": "curso_ingresso_ufrj",
#     "cursoAtual": "curso_atual",
#     "situacaoMatriculaAtual": "situacao_matricula",
#     "periodoIngressoCursoAtual": "periodo_ingresso_curso_atual",
#     "periodosIntegralizados": "periodos_integralizados",
#     "cargaHorariaAcumulada": "carga_horaria_acumulada",
# }
# df_completo.rename(columns=colunas, inplace=True)

In [190]:
df_completo.to_csv('data/silver/arquivo_anonimizado_v2.csv', index=False)

# Realizando Criação das Tabelas


## Dimenssão Aluno


In [242]:
colunas_aluno = ["sk_d_aluno", "nome_completo", "matricula_dre", "sexo", "data_nascimento"]

In [243]:
d_aluno = df_completo[colunas_aluno[1:]]

d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)
d_aluno

,nome_completo,matricula_dre,sexo,data_nascimento
0,Laura Rezende,0,M,06/02/1982
1,Diego Cardoso,1,M,05/06/1981
2,Isabel Porto,2,M,24/08/1980
3,Ian Gonçalves,3,M,18/08/1978
4,Emanuelly Aragão,4,M,02/05/1981
...,...,...,...,...
2700,Joana Cardoso,2341,M,22/11/1997
2701,Isabella da Mata,2345,M,17/11/1999
2702,Beatriz Moreira,2399,M,10/02/2000
2703,Julia Monteiro,2435,M,05/02/2002


In [244]:
d_aluno["sk_d_aluno"] = d_aluno.apply(
    lambda x: hashlib.sha256(str(x["matricula_dre"]).encode()).hexdigest(), axis=1
)
d_aluno = d_aluno.reindex(columns=colunas_aluno)
d_aluno.to_csv("data/gold/d_aluno.csv", index=False)
d_aluno

,sk_d_aluno,nome_completo,matricula_dre,sexo,data_nascimento
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,Laura Rezende,0,M,06/02/1982
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Diego Cardoso,1,M,05/06/1981
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Isabel Porto,2,M,24/08/1980
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Ian Gonçalves,3,M,18/08/1978
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Emanuelly Aragão,4,M,02/05/1981
...,...,...,...,...,...
2700,27d73d98c36f3dc0af6a78ae35b7ac66cbb5b379984500...,Joana Cardoso,2341,M,22/11/1997
2701,38083c7ee9121e17401883566a148aa5c2e2d55dc53bc4...,Isabella da Mata,2345,M,17/11/1999
2702,3b25e0d5df8f8001725cde4476e96f163548426c2ac14f...,Beatriz Moreira,2399,M,10/02/2000
2703,1d234b0fe9253354bebf1a88ba68514cbcf72e2d853ba1...,Julia Monteiro,2435,M,05/02/2002


## Dimenssão Disciplina


In [245]:
colunas_disciplina = ["sk_d_disciplina", "cod_disciplina", "nome_disciplina"]

In [246]:
d_disciplina = df_completo[colunas_disciplina[1:]].dropna()
d_disciplina = d_disciplina.drop_duplicates().reset_index(drop=True)
d_disciplina

,cod_disciplina,nome_disciplina
0,MAA123,Algebra para Informatica
1,MAB111,Fund da Computação Digital
2,MAB120,Computacao para Informatica
3,MAE111,Cálculo Infinitesimal I
4,MAE115,Cálculo Vetorial e G Analitica
...,...,...
1467,FIW481,Introducao a Termodinamica
1468,FIW482,Lab de Fisica Moderna I
1469,FIW590,Física Moderna B
1470,FIW591,Topicos de Eletromagnetismo


In [247]:
d_disciplina["sk_d_disciplina"] = d_disciplina.apply(
    lambda x: hashlib.sha256(
        str(x["cod_disciplina"] + x["nome_disciplina"]).encode()
    ).hexdigest(),
    axis=1,
)
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)
d_disciplina.to_csv("data/gold/d_disciplina.csv", index=False)
d_disciplina

,sk_d_disciplina,cod_disciplina,nome_disciplina
0,26ae25e81ccc85aecdad201a410a09485d415acd4daf6e...,MAA123,Algebra para Informatica
1,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,MAB111,Fund da Computação Digital
2,09e5060ee94ea2b9604e23295927e68d40d481a362d81e...,MAB120,Computacao para Informatica
3,63637e47a78400c87bb1b82da2fc5b6c00e78c0c77c779...,MAE111,Cálculo Infinitesimal I
4,6f904cf605935af92e93713c9eead620c9c92cd971dfc4...,MAE115,Cálculo Vetorial e G Analitica
...,...,...,...
1467,130782568a6092ad659a42f8ef804e17bb94a23f3e9685...,FIW481,Introducao a Termodinamica
1468,db788f41dbfd18af00a6f0dc974f1dabd113bc8d78c70a...,FIW482,Lab de Fisica Moderna I
1469,baf69a8104095b78e3311c071505a6ec05842794c4de3d...,FIW590,Física Moderna B
1470,5236e3878333e2a665b01166a10f59fea6e8bc01b471a3...,FIW591,Topicos de Eletromagnetismo


## Dimenssão Periodo


In [248]:
colunas_periodo = ["sk_d_periodo", "ano", "semestre", "periodo"]

In [249]:
d_periodo = df_completo[colunas_periodo[1:]]
d_periodo = d_periodo.drop_duplicates().reset_index(drop=True)
d_periodo

,ano,semestre,periodo
0,2000,1,2000/1
1,2000,2,2000/2
2,2001,1,2001/1
3,2001,2,2001/2
4,2002,1,2002/1
...,...,...,...
65,2018,0,2018/0
66,2019,0,2019/0
67,2013,3,2013/3
68,2014,3,2014/3


In [250]:
d_periodo["sk_d_periodo"] = d_periodo.apply(
    lambda x: hashlib.sha256(str(x["ano"] + x["semestre"]).encode()).hexdigest(), axis=1
)
d_periodo = d_periodo.reindex(columns=colunas_periodo)
d_periodo.to_csv("data/gold/d_periodo.csv", index=False)
d_periodo

,sk_d_periodo,ano,semestre,periodo
0,fc82267b45dcbf8db9e4ec10055373cfff41efcbc5b83a...,2000,1,2000/1
1,29433eae6f7f1308d9799275f3a90a0afe1fef0e1818a7...,2000,2,2000/2
2,e4b8f2da99fd9407c758cd306f846562424ec5c38d3202...,2001,1,2001/1
3,b6167a59dbfdca3d716a326a015dd8a00f417590eb34e2...,2001,2,2001/2
4,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,2002,1,2002/1
...,...,...,...,...
65,f45359f69577b469e8ada76cd6e045b302a4fcfaa7e0d1...,2018,0,2018/0
66,8ad00ecd5eb3a6ca42f90cc776b009968ec5865f2d37b3...,2019,0,2019/0
67,fa103adf3e582291297b8330060eace6bc1a14f8737cf2...,2013,3,2013/3
68,10071c57cf3b2135572815c0243a079fc18936a1259c3d...,2014,3,2014/3


## Dimenssão Curso


In [251]:
colunas_curso = ["sk_d_curso", "curso_ingresso_ufrj", "curso_atual"]

In [252]:
d_curso = df_completo[colunas_curso[1:]]
d_curso = d_curso.drop_duplicates().reset_index(drop=True)
d_curso

,curso_ingresso_ufrj,curso_atual
0,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,Bacharelado em Informática,Bacharelado em Informática
2,Bacharelado em Informática,Mat Aplicada - Computação Científica
3,Bacharelado em Informática,Licenciatura em Matemática
4,Matemática,Bacharelado em Ciência da Computação
...,...,...
74,Licenciatura em Física,Bacharelado em Ciência da Computação
75,Ciências Matemáticas e da Terra,Ciência da Computação
76,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação
77,Bacharelado em Ciência da Computação,Licenciatura em Geografia


In [253]:
d_curso["sk_d_curso"] = d_curso.apply(
    lambda x: hashlib.sha256(
        str(x["curso_ingresso_ufrj"] + x["curso_atual"]).encode()
    ).hexdigest(),
    axis=1,
)
d_curso = d_curso.reindex(columns=colunas_curso)
d_curso.to_csv("data/gold/d_curso.csv", index=False)
d_curso

,sk_d_curso,curso_ingresso_ufrj,curso_atual
0,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,dbde55e27224a8c832119eba0c54cb25a32bf6f6417dd2...,Bacharelado em Informática,Bacharelado em Informática
2,0367c21aa27e715610d936288c8fb811fc6dd378b050b5...,Bacharelado em Informática,Mat Aplicada - Computação Científica
3,cb369a9ae66bcd4659f91ed5d88c2a7674ca54645d641a...,Bacharelado em Informática,Licenciatura em Matemática
4,5f20305be16f0143bc9a1bcc213bc104dd93a64e5ad031...,Matemática,Bacharelado em Ciência da Computação
...,...,...,...
74,b04e50e1e81b07e4bba26b99505e3a21fd9160bdd6c664...,Licenciatura em Física,Bacharelado em Ciência da Computação
75,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,Ciências Matemáticas e da Terra,Ciência da Computação
76,a54d2fed0213e2b06b6fb51bf134c8f5920420505116c3...,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação
77,2d37b7f8c6308826f3cd805e314628baf4dede1e70bb0a...,Bacharelado em Ciência da Computação,Licenciatura em Geografia


## Fato Desempenho Academico


In [254]:
map_disciplina = {
    'Aprovado': 'Aprovado',
    'Reprovadomedia' : 'Reprovado',
    'Reprfalta/media': 'Reprovado',
    'Reprovadofaltas': 'Reprovado',
    'Grauincompleto': 'Reprovado',
}

In [255]:
f_desempenho_academico = df_completo.drop(
    [
        "periodo_ingresso_curso_atual",
        "periodos_integralizados",
        "carga_horaria_acumulada",
        "situacao_matricula"
    ],
    axis=1,
)
f_desempenho_academico['situacao_disciplina'] = f_desempenho_academico['situacao_disciplina'].map(map_disciplina)
f_desempenho_academico.head(1)

,periodo,cr_acumulado,matricula_dre,cr_periodo,cod_disciplina,nome_disciplina,grau_disciplina,situacao_disciplina,periodo_ingresso_ufrj,curso_ingresso_ufrj,...,codCursoAtual,manutencaoDeVinculo,formaIngresso,notaEnem,modalidadeCota,data_nascimento,sexo,nome_completo,ano,semestre
0,2000/1,3.9,0,3.8,MAA123,Algebra para Informatica,021,Reprovado,2000/1,Bacharelado em Informática,...,3101070000,0,Vestibular,NaN,Ampla Concorrência,06/02/1982,M,Laura Rezende,2000,1


In [256]:
# D_PERIODO
f_desempenho_academico = pd.merge(
    f_desempenho_academico, d_periodo, on=["ano", "semestre", "periodo"], how="left"
)
f_desempenho_academico.drop(["ano", "semestre", "periodo"], axis=1, inplace=True)

In [257]:
# D_CURSO
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_curso,
    on=["curso_ingresso_ufrj", "curso_atual"],
    how="left",
)
f_desempenho_academico.drop(
    ["curso_ingresso_ufrj", "curso_atual"], axis=1, inplace=True
)

In [258]:
# D_ALUNO
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_aluno,
    on=["nome_completo", "sexo", "matricula_dre", "data_nascimento"],
    how="left",
)
f_desempenho_academico.drop(
    ["nome_completo", "sexo", "matricula_dre"], axis=1, inplace=True
)

In [259]:
# D_DISCIPLINA
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_disciplina,
    on=["cod_disciplina", "nome_disciplina"],
    how="left",
)
f_desempenho_academico.drop(["cod_disciplina", "nome_disciplina"], axis=1, inplace=True)

In [260]:
f_desempenho_academico.drop(["periodo_ingresso_ufrj"], axis=1, inplace=True)

In [261]:
f_desempenho_academico.sample(5).T

,9012,3147,5225,29326,73522
cr_acumulado,2.6,0.3,1.6,5.1,5.4
cr_periodo,2.7,0.3,0.1,4.6,6.1
grau_disciplina,022,000,000,050,
situacao_disciplina,Reprovado,Reprovado,Reprovado,Aprovado,Aprovado
codCursoIngresso,3101020000,3101020000,3101020000,3103010100,3171010000
codCursoAtual,3101070000,3101020000,3101020000,3101070000,3109000100
manutencaoDeVinculo,0,0,0,0,0
formaIngresso,Vestibular,Transferência,Vestibular,Vestibular,SiSU - Sistema de Seleção Unificada
notaEnem,NaN,NaN,NaN,NaN,618.96
modalidadeCota,Ampla Concorrência,Ampla Concorrência,Ampla Concorrência,Ampla Concorrência,Ampla Concorrência


In [262]:
f_desempenho_academico.to_csv("data/gold/f_desempenho_academico.csv", index=False)

## Fato Situação Matricula Aluno Final

In [286]:
f_matricula_aluno = df_completo[df_completo['semestre'] != '0']

In [287]:
f_matricula_aluno = f_matricula_aluno.drop(
    [
        "cod_disciplina",
        "nome_disciplina",
        "grau_disciplina",
        "situacao_disciplina",
        "cr_periodo",
    ],
    axis=1,
)

In [288]:
indices_max_periodo = f_matricula_aluno.groupby('matricula_dre')['periodo'].idxmax()

f_matricula_aluno = f_matricula_aluno.loc[indices_max_periodo]

In [292]:
# D_PERIODO
f_matricula_aluno = pd.merge(
    f_matricula_aluno, d_periodo, on=["ano", "semestre", "periodo"], how="left"
)
f_matricula_aluno.drop(["ano", "semestre", "periodo"], axis=1, inplace=True)

In [290]:
# D_ALUNO
f_matricula_aluno = pd.merge(
    f_matricula_aluno,
    d_aluno,
    on=["nome_completo", "sexo", "matricula_dre", "data_nascimento"],
    how="left",
)
f_matricula_aluno.drop(
    ["nome_completo", "sexo", "matricula_dre"], axis=1, inplace=True
)

In [291]:
# D_CURSO
f_matricula_aluno = pd.merge(
    f_matricula_aluno,
    d_curso,
    on=["curso_ingresso_ufrj", "curso_atual"],
    how="inner",
)
f_matricula_aluno.drop(
    ["curso_ingresso_ufrj", "curso_atual"], axis=1, inplace=True
)

In [295]:
f_matricula_aluno.to_csv("data/gold/f_matricula_aluno.csv", index=False)